In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_posttypeid \
                , x.q_acceptedanswerid \
                , x.q_parentid \
                , x.q_creationdate \
                , x.q_score \
                , x.q_viewcount \
                , x.q_owneruserid \
                , x.q_title \
                , x.q_tags \
                , x.q_answercount \
                , x.q_commentcount \
                , x.q_favoritecount \
                , x.q_closeddate \
                , b.id               as a_id \
                , b.posttypeid       as a_posttypeid \
                , b.acceptedanswerid as a_acceptedanswerid \
                , b.parentid         as a_parentid \
                , b.creationdate     as a_creationdate \
                , b.score            as a_score \
                , b.viewcount        as a_viewcount \
                , b.owneruserid      as a_owneruserid \
                , b.title            as a_title \
                , b.tags             as a_tags \
                , b.answercount      as a_answercount \
                , b.commentcount     as a_commentcount \
                , b.favoritecount    as a_favoritecount \
                , b.closeddate       as a_closeddate \
  from ( \
           select a.id               as q_id \
                , a.posttypeid       as q_posttypeid \
                , a.acceptedanswerid as q_acceptedanswerid \
                , a.parentid         as q_parentid \
                , a.creationdate     as q_creationdate \
                , a.score            as q_score \
                , a.viewcount        as q_viewcount \
                , a.owneruserid      as q_owneruserid \
                , a.title            as q_title \
                , a.answercount      as q_answercount \
                , a.commentcount     as q_commentcount \
                , a.favoritecount    as q_favoritecount \
                , a.closeddate       as q_closeddate \
                , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags \
           from posts a \
           where a.creationdate between '2013-11-01' and '2023-12-01' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
       )   x \
        , posts b \
where b.parentid = x.q_id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_da = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_favoritecount'
, 'q_closeddate' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_favoritecount'
, 'a_closeddate' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_da.pkl', 'wb') as f:
    pickle.dump(df_da, f)

In [3]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select a.creationdate \
     , a.id \
     , a.posttypeid \
     , a.title \
     , a.owneruserid \
     , b.body \
     , case when c.id is null then 0 else 1 end as answered_yn \
  from posts as a \
     inner join postsbody b \
     on a.id = b.id \
     left outer join vw_posts_get_answer as c \
     on a.id = c.id \
where a.creationdate between '2013-11-01' and '2023-12-01' \
  ; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [6]:
df_body = pd.DataFrame(rows, columns = [
  'creationdate' 
, 'id' 
, 'posttypeid'
, 'title'
, 'owneruserid'
, 'body' 
, 'answered_yn'
])

In [7]:
# 전체 데이터셋 
with open('../../data/df_body.pkl', 'wb') as f:
    pickle.dump(df_body, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select id, creationdate, displayname, lastaccessdate \
  from users \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_user = pd.DataFrame(rows, columns = [
  'id' 
, 'creationdate' 
, 'displayname'
, 'lastaccessdate'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_user.pkl', 'wb') as f:
    pickle.dump(df_user, f)

In [8]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select to_char(p.creationdate, 'yyyy') as year, p.id, count(a.id) as cnt \
  from posts p \
     left join posts a \
     on p.id = a.parentid \
     and a.posttypeid = '2' \
     and a.creationdate - p.creationdate <= '30 days' \
 where ((p.creationdate between '2023-11-01' and '2023-12-01') \
 	 or (p.creationdate between '2022-11-01' and '2022-12-01') \
 	 or (p.creationdate between '2021-11-01' and '2021-12-01') \
 	 or (p.creationdate between '2020-11-01' and '2020-12-01') \
 	 or (p.creationdate between '2019-11-01' and '2019-12-01') \
 	 ) \
   and p.posttypeid ='1' \
  group by p.id \
  ; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [9]:
df_answer_cnt = pd.DataFrame(rows, columns = [
  'year' 
, 'id'
, 'cnt' 
])

In [10]:
# 전체 데이터셋 
with open('../../data/df_answer_cnt.pkl', 'wb') as f:
    pickle.dump(df_answer_cnt, f)

In [9]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xx.year, xx.usr_id, count(xx.q_id) as cnt \
  from ( \
 select x.usr_id, x.usr_creationdate, x.year, p.id as q_id, p.creationdate as q_date \
   from ( \
  	select u.id as usr_id, u.creationdate as usr_creationdate, m.year \
     from users u \
        , ma_year m \
 	where to_char(u.creationdate, 'yyyy') <= m.year \
   	  and m.year between '2013' and '2023' \
      and u.id >1 \
  	) x \
   left join posts p \
   on x.usr_id = p.owneruserid \
   and p.posttypeid = '1' \
   and x.year = to_char(p.creationdate, 'yyyy') \
) xx \
group by xx.year, xx.usr_id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [10]:
df_usr_q_cnt = pd.DataFrame(rows, columns = [
  'year' 
, 'usr_id'
, 'cnt' 
])

In [11]:
# 전체 데이터셋 
with open('../../data/df_usr_q_cnt.pkl', 'wb') as f:
    pickle.dump(df_usr_q_cnt, f)

In [12]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xxx.year, avg(xxx.cnt) as avg cnt \
  from ( \
		  select xx.year, xx.usr_id, count(xx.q_id) as cnt \
		  from (select x.usr_id, x.usr_creationdate, x.year, p.id as q_id, p.creationdate as q_date \
		   from ( \
		  	select u.id as usr_id, u.creationdate as usr_creationdate, m.year \
		     from users u \
		        , ma_year m \
		 	where to_char(u.creationdate, 'yyyy') <= m.year \
		   	  and m.year between '2013' and '2023' \
		      and u.id >1 \
		  	) x \
		   left join posts p \
		   on x.usr_id = p.owneruserid \
		   and p.posttypeid = '2' \
		   and x.year = to_char(p.creationdate, 'yyyy') \
		) xx \
		group by xx.year, xx.usr_id \
  ) xxx \
 group by xxx.year \
  ;              "
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [13]:
df_usr_a_cnt = pd.DataFrame(rows, columns = [
  'year' 
, 'usr_id'
, 'cnt' 
])

In [14]:
# 전체 데이터셋 
with open('../../data/df_usr_a_cnt.pkl', 'wb') as f:
    pickle.dump(df_usr_a_cnt, f)

: 

In [ ]:
# create table tt_year_from_signup_qna as
# select x.usr_id
#      , p.id as q_id
#      , ceil(DATE_PART('day',p.creationdate::timestamp-x.usr_creationdate::timestamp)/365) as yeardiff
#      , p.creationdate, x.usr_creationdate
#   from (
#         select u.id as usr_id, u.creationdate as usr_creationdate
#           from users u
#          where u.id >1
#         ) x
#         left join posts p
#           on x.usr_id = p.owneruserid
#          and p.posttypeid in( '1', '2')
# ;


In [3]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select yeardiff, count(q_id) as cnt \
  from public.tt_year_from_signup_a \
 group by yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [4]:
df_year_from_signup_a = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'cnt' 
])


In [5]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_a.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_a, f)

In [6]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select yeardiff, count(q_id) as cnt \
  from public.tt_year_from_signup_qna \
 group by yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [7]:
df_year_from_signup_qna = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'cnt' 
])


In [8]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_qna.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_qna, f)

In [9]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select yeardiff, count(q_id) as cnt \
  from public.tt_year_from_signup_q \
 group by yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [12]:
df_year_from_signup_q = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'cnt' 
])


In [13]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_q.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_q, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.yeardiff, avg(x.cnt) as mean_cnt , median(x.cnt) as med_cnt \
  from ( \
select a.yeardiff, a.usr_id, count(a.q_id) as cnt \
  from public.tt_year_from_signup_qna a \
 where a.yeardiff >=0 \
 group by a.yeardiff, a.usr_id \
  ) x \
 group by x.yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_from_signup_q_mm = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'mean_cnt' 
, 'med_cnt'
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_q_mm.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_q_mm, f)